# Project: NeoExoSphyre Detecting near Earth objects

### Import modules

In [ ]:
import re
import requests
import pathlib
import sqlite3

# Define the directory and name of the NEODyS data
raw_data_dir = pathlib.Path("raw_data/")
raw_data_file = pathlib.Path("neodys.cat")

raw_data_filepath = raw_data_dir / raw_data_file
print(f"Our NEODyS file path: {raw_data_filepath}")

pathlib.Path.mkdir(raw_data_dir, exist_ok=True)

### Get the current number of known NEOs

In [2]:
http_response = requests.get("https://newton.spacedys.com/neodys/index.php?pc=1.0")
html_content = http_response.content

# Extract the number of NEOs from a specific HTML position, using a regular expression. The
# number is displayed in bold like "[...] <b>1000 objects</b> in the NEODys [...]"
neodys_nr_neos = int(re.findall(r"<b>(.*?) objects</b> in the NEODyS", str(html_content))[0])

In [3]:
print(f"Number of currently known NEOs: {neodys_nr_neos}")

Number of currently known NEOs: 37416


### Download the NEODyS file and store it

In [4]:
response = requests.get("https://newton.spacedys.com/~neodys2/neodys.cat")
download_file_path = pathlib.Path(raw_data_filepath)
with download_file_path.open(mode="wb+") as file_obj:
    file_obj.write(response.content)

In [5]:
# Set a placeholder dictionary where the data will be stored
neo_dict = []

# Open the NEODyS file. Ignore the header (first 6 rows) and iterate through the file row-wise.
# Read the content adn save it in the dictionary
with open(raw_data_filepath) as f_temp:
    neo_data = f_temp.readlines()[6:]
    for neo_data_line_f in neo_data:
        neo_data_line = neo_data_line_f.split()
        neo_dict.append(
            {
                "Name": neo_data_line[0].replace("'", ""),
                "Epoch_MJD": float(neo_data_line[1]),
                "SemMajAxis_AU": float(neo_data_line[2]),
                "Ecc_": float(neo_data_line[3]),
                "Incl_deg": float(neo_data_line[4]),
                "LongAscNode_deg": float(neo_data_line[5]),
                "ArgP_deg": float(neo_data_line[6]),
                "MeanAnom_deg": float(neo_data_line[7]),
                "AbsMag_": float(neo_data_line[8]),
                "SlopeParamG_": float(neo_data_line[9]),
            }
        )

In [6]:
print(f"Does the file contain the same number of NEOs as the NEODyS website? \n"
      f"{'>Yes' if len(neo_dict) == neodys_nr_neos else '>No'}")

Does the file contain the same number of NEOs as the NEODyS website? 
>Yes


### Let's take a look!

In [7]:
neo_dict[:2]

[{'Name': '433',
  'Epoch_MJD': 60000.0,
  'SemMajAxis_AU': 1.458129137561811,
  'Ecc_': 0.22278187193561982,
  'Incl_deg': 10.827822059970396,
  'LongAscNode_deg': 304.2870240036129,
  'ArgP_deg': 178.92698564290663,
  'MeanAnom_deg': 110.77766380976027,
  'AbsMag_': 10.91,
  'SlopeParamG_': 0.46},
 {'Name': '719',
  'Epoch_MJD': 60600.0,
  'SemMajAxis_AU': 2.636157149048896,
  'Ecc_': 0.5467795293144841,
  'Incl_deg': 11.575260341822329,
  'LongAscNode_deg': 183.8571635400628,
  'ArgP_deg': 156.2155185178333,
  'MeanAnom_deg': 148.45067666976948,
  'AbsMag_': 15.54,
  'SlopeParamG_': 0.15}]

### Now we create the NEODyS SQLite database for our future project work

In [8]:
database_dir = pathlib.Path("../databases/neos/")
database_file = pathlib.Path("neodys.db")
database_filepath = database_dir / database_file

# Create the directory
pathlib.Path.mkdir(database_dir, parents=True, exist_ok=True)

# Establish a connection to the database and set a cursor
neodys_db_con = sqlite3.connect(database_filepath)
neodys_db_cur = neodys_db_con.cursor()

### Now we create the NEODyS SQLite database for our future project work

In [9]:
neodys_db_cur.execute(
    "CREATE TABLE IF NOT EXISTS main(Name TEXT PRIMARY KEY, "
    "Epoch_MJD FLOAT, "
    "SemMajAxis_AU FLOAT, "
    "Ecc_ FLOAT, "
    "Incl_deg FLOAT, "
    "LongAscNode_deg FLOAT, "
    "ArgP_deg FLOAT, "
    "MeanAnom_deg FLOAT, "
    "AbsMag_ FLOAT, "
    "SlopeParamG_ FLOAT)"
)
neodys_db_con.commit()

# Insert the raw data into the database
neodys_db_cur.executemany(
    "INSERT OR IGNORE INTO main(Name, "
    "Epoch_MJD, "
    "SemMajAxis_AU, "
    "Ecc_, "
    "Incl_deg, "
    "LongAscNode_deg, "
    "ArgP_deg, "
    "MeanAnom_deg, "
    "AbsMag_, "
    "SlopeParamG_) "
    "VALUES (:Name, "
    ":Epoch_MJD, "
    ":SemMajAxis_AU, "
    ":Ecc_, "
    ":Incl_deg, "
    ":LongAscNode_deg, "
    ":ArgP_deg, "
    ":MeanAnom_deg, "
    ":AbsMag_, "
    ":SlopeParamG_)",
    neo_dict,
)
neodys_db_con.commit()

### Add new columns in the main table

In [10]:
for col_name in ["Aphel_AU", "Perihel_AU"]:
    
    # SQL Query for creating new columns
    sql_col_create = f"ALTER TABLE main ADD COLUMN {col_name} FLOAT"

    # Try to create a new column. If it exists an sqlite3.OperationalError will raise. Pass this error.
    try:
        neodys_db_cur.execute(sql_col_create)
        neodys_db_con.commit()
    except sqlite3.OperationalError:
        pass

### Get orbital elements to compute the derived parameters

In [11]:
neodys_db_cur.execute("SELECT Name, SemMajAxis_AU, Ecc_ FROM main")

# Fetch the data
_neo_data = neodys_db_cur.fetchall()

# Iterate throuh the results, compute the derived elements and put them in a list of
# dicitionaries
_neo_deriv_param_dict = []
for _neo_data_line_f in _neo_data:
    _neo_deriv_param_dict.append(
        {
            "Name": _neo_data_line_f[0],
            "Aphel_AU": (1.0 + _neo_data_line_f[2]) * _neo_data_line_f[1],
            "Perihel_AU": (1.0 - _neo_data_line_f[2]) * _neo_data_line_f[1],
        }
    )

# Insert the data into the main table
neodys_db_cur.executemany(
    "UPDATE main SET Aphel_AU = :Aphel_AU, Perihel_AU = :Perihel_AU "
    "WHERE Name = :Name",
    _neo_deriv_param_dict,
)
neodys_db_con.commit()

In [12]:
neodys_db_con.close()

In [13]:
# Import modules
import pathlib
import sqlite3

### Now we create the NEODyS SQLite database for our future project work

In [14]:
database_dir = pathlib.Path("../databases/neos/")
database_file = pathlib.Path("neodys.db")
database_filepath = database_dir / database_file

# Create the directory
pathlib.Path.mkdir(database_dir, parents=True, exist_ok=True)

# Establish a connection to the database and set a cursor
neodys_db_con = sqlite3.connect(database_filepath)
neodys_db_cur = neodys_db_con.cursor()

In [15]:
def neo_class(sem_maj_axis_au: float,
              peri_helio_au: float,
              ap_helio_au: float) -> str:
    """Classify the NEO based on the orbital parameters.
    Depending on the semi-major axis, perihelion and / or aphelion a NEO can be classified as an
    Amor, Apollo, Aten or Atira.
    Parameters
    ----------
    sem_maj_axis_au : float
        Semi-major axis of the NEO. Given in AU
    peri_helio_au : float
        Perihelion of the NEO. Given in AU
    ap_helio_au : float
        Aphelion of the NEO. Given in AU
    Returns
    -------
    neo_type : str
        NEO class / type.
    References
    ----------
    -1- Link to the NEO classifiction schema: https://cneos.jpl.nasa.gov/about/neo_groups.html
    """
    # Determine the NEO class in an extensive if-else statement
    if (sem_maj_axis_au > 1.0) & (1.017 < peri_helio_au < 1.3):
        neo_type = 'Amor'

    elif (sem_maj_axis_au > 1.0) & (peri_helio_au < 1.017):
        neo_type = 'Apollo'

    elif (sem_maj_axis_au < 1.0) & (ap_helio_au > 0.983):
        neo_type = 'Aten'

    elif (sem_maj_axis_au < 1.0) & (ap_helio_au < 0.983):
        neo_type = 'Atira'

    else:
        neo_type = 'Other'

    return neo_type

### Testing the function

In [16]:
amor1221 = [1.9191, 1.0832, 2.7550]
apollo1862 = [1.4702, 0.64699, 	2.2935]
aten2062 = [0.9668, 0.7901, 1.1434]
atira163693 = [0.7411, 0.5024, 0.9798]

assert "Amor" == neo_class(*amor1221)
assert "Apollo" == neo_class(*apollo1862)
assert "Aten" == neo_class(*aten2062)
assert "Atira" == neo_class(*atira163693)

In [17]:
def create_col(sqlitecon : sqlite3.Connection, 
               sqlitecur: sqlite3.Cursor,
               table: str,
               col_name: str,
               col_type: str) -> None:
    """
    Function to create new columns in tables.
    Parameters
    ----------
    sqlitecon : sqlite3.Connection
        Connection to the SQLite database.
    sqlitecur : sqlite3.Cursor
        Cursor that points to the SQLite database.
    sqlitetable : str
        Table name, where a new column shall be added.
    col_name : str
        Column name.
    col_type : str
        SQLite column type (FLOAT, INT, TEXT, etc.).
    """
    # Generic f-string that represents an SQLite command to alter a table (adding a new column
    # with its dtype).
    sql_col_create = f"ALTER TABLE {table} ADD COLUMN {col_name} {col_type}"

    # Try to create a new column. If is exists an sqlite3.OperationalError will raise. Pass
    # this error.
    try:
        sqlitecur.execute(sql_col_create)
        sqlitecon.commit()
    except sqlite3.OperationalError:
        pass

### Add a new column in the main table

In [18]:
create_col(sqlitecon=neodys_db_con,
           sqlitecur=neodys_db_cur,
           table="main",
           col_name="NEOClass",
           col_type="Text")

### Get the orbital elements to compute the NEO class

In [19]:
neodys_db_cur.execute("SELECT Name, SemMajAxis_AU, Perihel_AU, Aphel_AU FROM main")

# Fetch the data
neo_data = neodys_db_cur.fetchall()

# Iterate throuh the results, compute the NEO class and put in into the results
neo_class_param_dict = []
for neo_data_row in neo_data:
    neo_class_param_dict.append(
        {
            "Name": neo_data_row[0],
            "NEOClass": neo_class(
                sem_maj_axis_au=neo_data_row[1],
                peri_helio_au=neo_data_row[2],
                ap_helio_au=neo_data_row[3]
            )
        }
    )

# Insert the data into the main table
neodys_db_cur.executemany(
    "UPDATE main SET NEOClass = :NEOClass "
    "WHERE Name = :Name",
    neo_class_param_dict,
)
neodys_db_con.commit()

In [20]:
neodys_db_con.close()